In [103]:
import pandas as pd 
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.deterministic import CalendarFourier, DeterministicProcess
from sklearn.preprocessing import StandardScaler

In [54]:
path2 = 'energy_dataset.csv'
csv2 = pd.read_csv(path2)
df_energy = pd.DataFrame(csv2)
df_energy['time'] = pd.to_datetime(df_energy['time'], utc=True)
df = df_energy.set_index("time")[["total load actual"]].copy()

In [ ]:
def visual(df, time_col='time', val_col ='total load actual',  n_points=None, title='Потребление - время', label = 'Потребление', xlabel='Дата', ylabel='МВт' ):
    if time_col not in df.columns:
        time_data = df.index
    else:
        time_data = df[time_col]

    if n_points is None:
        value_data = df[val_col]
    else:
        time_data = time_data[:n_points]
        value_data = df[val_col][:n_points]

    plt.figure(figsize=(18,5))
    plt.plot(time_data, value_data, label=label)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()
    plt.show()

In [82]:
def corralelogramm(df, lags_acf = 30, lags_pacf = 30):
    plot_acf(df.dropna(), lags = lags_acf)
    plt.title('АКФ')
    plt.show()
    plot_pacf(df.dropna(), lags = lags_pacf)
    plt.title("ЧАКФ")
    plt.show()

In [59]:
def diff(df, lag = 1):
    df_diff = df.diff(lag).dropna()
    return df_diff

In [83]:
def create_exog_fourier(df, D=2, W=4, ME=4, seasonal=False, order = 0):
    df.index = pd.to_datetime(df.index).tz_localize(None)
    fourier_daily = CalendarFourier(freq='D', order=D)
    fourier_week  = CalendarFourier(freq='W', order=W)
    fourier_month  = CalendarFourier(freq='ME', order=ME)
    dp = DeterministicProcess(
        index=df.index,
        constant=True,
        order=order,
        seasonal=seasonal,                
        additional_terms=[fourier_daily, fourier_week, fourier_month],
        drop=True,
    )
    df_fourier = dp.in_sample()
    return df_fourier


In [ ]:
def create_exog_calendar(df, freq='h'):
    df_features = pd.DataFrame(index=df.index).asfreq(freq) 
    df_features['is_weekend'] = (df.index.dayofweek >= 5).astype(int)
    df_features['month'] = df.index.month
    df_features['is_night'] = ((df.index.hour >= 4) & (df.index.hour <= 7)).astype(int)
    df_features['is_evening'] = ((df.index.hour >= 20) | (df.index.hour == 0)).astype(int)
    return df_features

In [99]:
def concat_and_create_exog_fourier_for_weekend(df_fourier, df_features, freq='h'):
    exog = pd.concat([df_fourier, df_features], axis=1)
    exog = exog.dropna()        
    exog['hour'] = exog.index.hour
    hour_dummies = pd.get_dummies(exog['hour'], prefix=freq, drop_first=True)
    fourier_week_cols = [c for c in exog.columns if 'freq=W' in str(c) or 'W-SUN' in str(c)] 
    for c in fourier_week_cols:
        exog[c + '_wkend'] = exog[c] * exog['is_weekend']
    return exog


In [ ]:
df_fourier = create_exog_fourier(df)
df_features = create_exog_calendar(df)
exog = concat_and_create_exog_fourier_for_weekend(df_fourier, df_features)

In [113]:
def train_test_split_with_exog(df, exog, split=0.8):
    df = df.loc[exog.index]
    train_size = int(len(df) * 0.8)
    train, test = df.iloc[:train_size], df.iloc[train_size:]
    exog_train, exog_test = exog.iloc[:train_size], exog.iloc[train_size:]
    return train, test, exog_train, exog_test 

In [108]:
def exog_scaler(exog_train, exog_test):
    sc = StandardScaler()
    exog_train_s = pd.DataFrame(sc.fit_transform(exog_train), index=exog_train.index, columns=exog_train.columns)
    exog_test_s  = pd.DataFrame(sc.transform(exog_test), index=exog_test.index, columns=exog_test.columns)
    return exog_train_s, exog_test_s

In [117]:
train, test, exog_train, exog_test  = train_test_split_with_exog(df, exog)
ex_tr, ex_test = exog_scaler(exog_train, exog_test)